In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F

# Hyperparameters
num_epochs = 30
num_classes = 2
learning_rate = 0.002

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
torch.cuda.manual_seed(789)

In [3]:
df = pd.read_csv("data/creditcard.csv", sep=",", index_col=None)
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1, 1))
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,1.641931,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.350151,0
284803,1.641952,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,-0.254117,0
284804,1.641974,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,-0.081839,0
284805,1.641974,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.313249,0
284806,1.642058,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,0.514355,0


In [5]:
anomalies = df[df["Class"] == 1]
normal = df[df["Class"] == 0]

anomalies.shape, normal.shape

((492, 31), (284315, 31))

In [6]:
for f in range(0, 20):
    normal = normal.iloc[np.random.permutation(len(normal))]
    

data_set = pd.concat([normal[:10000], anomalies])

x_train, x_test = train_test_split(data_set, test_size = 0.4, random_state = 42)

x_train = x_train.sort_values(by=['Time'])
x_test = x_test.sort_values(by=['Time'])

y_train = x_train["Class"]
y_test = x_test["Class"]

x_train.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
15,-1.996330,-0.752417,0.345485,2.057323,-1.468643,-1.158394,-0.077850,-0.608581,0.003603,-0.436167,...,0.499625,1.353650,-0.256573,-0.065084,-0.039124,-0.087086,-0.180998,0.129394,-0.289300,0
70,-1.995614,1.006589,-0.071105,0.347614,1.329684,-0.193240,0.155418,0.008574,0.146538,0.103844,...,-0.020239,-0.018881,-0.120966,0.027382,0.593864,-0.334688,0.021368,0.015080,-0.081479,0
152,-1.994583,-0.343711,1.118615,1.293386,0.064762,0.023481,-0.992741,0.695814,-0.064809,-0.373640,...,-0.265392,-0.700550,-0.014280,0.330615,-0.165442,0.072342,0.244882,0.098048,-0.344114,0
172,-1.994267,-0.591933,0.111273,0.699126,-1.536074,1.193208,0.648896,0.796706,0.016904,0.789664,...,0.205964,1.216195,0.093396,-0.900310,-0.423966,-0.607857,0.017705,-0.076746,-0.297456,0
264,-1.992645,0.066061,0.876263,1.093993,1.556151,-0.072318,-0.671902,0.569707,-0.143791,-0.609469,...,0.159757,0.497797,-0.043266,0.397923,-0.358151,-0.186331,0.135739,0.098840,-0.259994,0
346,-1.991192,-0.471796,0.523169,1.948967,0.995503,0.379069,-0.577466,0.521413,-0.128940,-0.704962,...,-0.099422,-0.287139,0.151288,0.490367,-0.725252,-0.741834,0.004784,-0.045977,-0.279465,0
416,-1.990224,1.073785,-0.170037,0.790656,1.128605,-0.585960,0.070807,-0.414937,0.057949,0.503324,...,0.100287,0.147604,-0.184878,-0.460811,0.423909,-0.318168,0.046111,0.046843,0.006598,0
588,-1.987296,-0.421336,0.845373,-0.180053,-1.194077,2.737800,3.293114,0.375299,0.660924,-0.319588,...,-0.309462,-0.836051,-0.053805,0.940155,-0.395774,-0.006256,0.060371,-0.013897,-0.316167,0
623,-1.986644,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,1.761758,1
628,-1.986517,-0.597236,0.897114,0.765018,-1.008934,0.491639,-0.059136,0.595702,0.110417,-0.068274,...,-0.039377,-0.208065,-0.222443,-0.842566,-0.003832,0.250814,-0.045118,0.061893,-0.313249,0


In [7]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

print("Shapes:\nx_train:%s\ny_train:%s\n" % (x_train.shape, y_train.shape))
print("x_test:%s\ny_test:%s\n" % (x_test.shape, y_test.shape))

Shapes:
x_train:(6295, 31)
y_train:(6295,)

x_test:(4197, 31)
y_test:(4197,)



## pytorch는 convolution 연산을 하기 위해서는 channel 차원이 중간에 오는 것이 좋음. 따라서 2차원 데이터 프레임 형태의 데이터는 (레이블 개수, 1, 데이터차원) 형태로 바꿔줘야함~!!!

In [8]:
def spatial_torch(tensor):
    tensor = tensor.permute(0, 2, 1)
    tensor = F.dropout2d(tensor, 0.05)
    tensor = tensor.permute(0, 2, 1)
    
    return tensor

In [9]:
x = torch.rand((30, 31, 32))
x

tensor([[[0.8827, 0.9027, 0.4851,  ..., 0.0377, 0.5924, 0.6405],
         [0.9423, 0.5575, 0.0964,  ..., 0.5745, 0.1785, 0.8549],
         [0.4517, 0.7442, 0.6832,  ..., 0.8773, 0.3190, 0.9750],
         ...,
         [0.6026, 0.9154, 0.4191,  ..., 0.6926, 0.0911, 0.9612],
         [0.6695, 0.4732, 0.9874,  ..., 0.1790, 0.6424, 0.6051],
         [0.1841, 0.1350, 0.2721,  ..., 0.3233, 0.3503, 0.3466]],

        [[0.5608, 0.7581, 0.4252,  ..., 0.9600, 0.9338, 0.4037],
         [0.1198, 0.9453, 0.1453,  ..., 0.7804, 0.9841, 0.3979],
         [0.6672, 0.5420, 0.1392,  ..., 0.5081, 0.2394, 0.6024],
         ...,
         [0.0620, 0.5377, 0.9772,  ..., 0.0231, 0.3591, 0.2685],
         [0.9898, 0.1415, 0.7535,  ..., 0.8853, 0.4718, 0.4563],
         [0.6761, 0.9408, 0.6521,  ..., 0.2030, 0.9845, 0.1400]],

        [[0.1151, 0.1739, 0.5951,  ..., 0.0011, 0.6051, 0.2484],
         [0.4411, 0.7450, 0.1381,  ..., 0.6807, 0.8456, 0.7403],
         [0.8498, 0.8849, 0.7730,  ..., 0.1015, 0.0496, 0.

In [10]:
x_sp = spatial_torch(x)
x_sp

tensor([[[0.9291, 0.9502, 0.5106,  ..., 0.0397, 0.6236, 0.6742],
         [0.9919, 0.5869, 0.1015,  ..., 0.6047, 0.1879, 0.8999],
         [0.4755, 0.7834, 0.7191,  ..., 0.9235, 0.3358, 1.0263],
         ...,
         [0.6343, 0.9636, 0.4412,  ..., 0.7290, 0.0958, 1.0118],
         [0.7048, 0.4981, 1.0394,  ..., 0.1885, 0.6762, 0.6370],
         [0.1938, 0.1421, 0.2864,  ..., 0.3403, 0.3687, 0.3648]],

        [[0.5904, 0.7980, 0.4476,  ..., 1.0105, 0.9830, 0.0000],
         [0.1261, 0.9951, 0.1530,  ..., 0.8215, 1.0359, 0.0000],
         [0.7023, 0.5705, 0.1465,  ..., 0.5348, 0.2520, 0.0000],
         ...,
         [0.0652, 0.5660, 1.0286,  ..., 0.0243, 0.3780, 0.0000],
         [1.0419, 0.1490, 0.7932,  ..., 0.9319, 0.4966, 0.0000],
         [0.7116, 0.9903, 0.6864,  ..., 0.2137, 1.0363, 0.0000]],

        [[0.0000, 0.1831, 0.6264,  ..., 0.0011, 0.6370, 0.2614],
         [0.0000, 0.7842, 0.1454,  ..., 0.7165, 0.8901, 0.7793],
         [0.0000, 0.9315, 0.8137,  ..., 0.1068, 0.0522, 0.

결과로 알 수 있듯이 특정 배치의 특정 열의 성분들이 전부 0으로 바뀜

In [11]:
class TCN(nn.Module):
    def __init__(self):
        super(TCN, self).__init__()
        
        self.conv_1 = nn.Conv1d(1, 128, kernel_size=2, dilation=1, padding=((2-1) * 1))
        self.conv_2 = nn.Conv1d(128, 128, kernel_size=2, dilation=2, padding=((2-1) * 2))
        self.conv_3 = nn.Conv1d(128, 128, kernel_size=2, dilation=4, padding=((2-1) * 4))
        self.conv_4 = nn.Conv1d(128, 128, kernel_size=2, dilation=8, padding=((2-1) * 8))
        self.dense_1 = nn.Linear(31*128  , 128)
        self.dense_2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv_1(x)
        x = x[:, :, :-self.conv_1.padding[0]]
        x = F.relu(x)
        x = spatial_torch(x)
        
        x = self.conv_2(x)
        x = x[:, :, :-self.conv_2.padding[0]]
        x = F.relu(x)
        x = spatial_torch(x)
        
        x = self.conv_3(x)
        x = x[:, :, :-self.conv_3.padding[0]]
        x = F.relu(x)
        x = spatial_torch(x)
        
        
        x = self.conv_4(x)
        x = x[:, :, :-self.conv_4.padding[0]]
        x = F.relu(x)
        x = spatial_torch(x)
        
        
        x = x.view(-1, 31*128)
        x = F.relu(self.dense_1(x))
        x = self.dense_2(x)
        return F.log_softmax(x, dim=1)

In [12]:
def train(model, device, x_train, y_train, criterion, optimizer, epoch, save_dir='TCN_CreditCard_PyTorch.ckpt'):
    total_step = len(x_train)
    
    x_train = torch.Tensor(x_train).unsqueeze(1).cuda().float()
    y_train = torch.Tensor(y_train).unsqueeze(1).cuda().long()

    x_train.to(device)
    y_train.to(device)

    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train.squeeze(1))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch {}/{}, Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    torch.save(model.state_dict(), save_dir)

In [13]:
from sklearn.metrics import roc_auc_score

def test(model, device, x_test, y_test):
    preds = []
    y_true = []
    
    # Set model to evaluation mode.
    model.eval()  
    with torch.no_grad():
        correct = 0
        total = 0
        
        x_test = torch.Tensor(x_test).unsqueeze(1).cuda().float()
        y_test = torch.Tensor(y_test).unsqueeze(1).cuda().long()
        
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        y_test = y_test.squeeze(1)
        outputs = model(x_test)
        _, predicted = torch.max(outputs.data, 1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()    
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = y_test.detach().cpu().numpy()
        for f in range(0, len(detached_label)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])

        print('Test Accuracy of the model: {:.2%}'.format(correct / total))

        preds = np.eye(num_classes)[preds]
        y_true = np.eye(num_classes)[y_true]  
        auc = roc_auc_score(np.round(preds), y_true)
        print("AUC: {:.2%}".format (auc))

In [14]:
model = TCN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Training phase

for epoch in range(0, num_epochs):
    train(model, device, x_train, y_train, criterion, optimizer, epoch)

Epoch 1/30, Loss: 0.6946
Epoch 2/30, Loss: 0.4355
Epoch 3/30, Loss: 0.3351
Epoch 4/30, Loss: 0.1899
Epoch 5/30, Loss: 0.1272
Epoch 6/30, Loss: 0.0872
Epoch 7/30, Loss: 0.0638
Epoch 8/30, Loss: 0.0696
Epoch 9/30, Loss: 0.0686
Epoch 10/30, Loss: 0.0709
Epoch 11/30, Loss: 0.0640
Epoch 12/30, Loss: 0.0528
Epoch 13/30, Loss: 0.0460
Epoch 14/30, Loss: 0.0434
Epoch 15/30, Loss: 0.0540
Epoch 16/30, Loss: 0.0445
Epoch 17/30, Loss: 0.0394
Epoch 18/30, Loss: 0.0413
Epoch 19/30, Loss: 0.0395
Epoch 20/30, Loss: 0.0406
Epoch 21/30, Loss: 0.0395
Epoch 22/30, Loss: 0.0395
Epoch 23/30, Loss: 0.0402
Epoch 24/30, Loss: 0.0369
Epoch 25/30, Loss: 0.0388
Epoch 26/30, Loss: 0.0369
Epoch 27/30, Loss: 0.0367
Epoch 28/30, Loss: 0.0382
Epoch 29/30, Loss: 0.0371
Epoch 30/30, Loss: 0.0363


In [15]:
test(model, device, x_test, y_test)

Test Accuracy of the model: 98.98%
AUC: 98.03%
